In [1]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
from IPython.display import clear_output

In [2]:
expiry_sp = 'BANKNIFTY24410'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [ ]:
# expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains('BANKNIFTY234'))].tradingsymbol.str.split('BANKNIFTY').str[1].str[:5].unique()
# expiry_march 

In [8]:
from tqdm import tqdm
start_dt = '2024-04-03'
end_dt = '2024-04-10'
time_frame = '3minute'

df_expiry_hist_data = pd.DataFrame()
strike_symbol_dict = dict(zip(expiry_march.instrument_token,expiry_march['strike_type'] ))
inst_expiry = expiry_march['instrument_token'].unique().tolist() 
for i in tqdm(inst_expiry):
    df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
    df['strike_type'] = strike_symbol_dict[i]
    df_expiry_hist_data = df_expiry_hist_data.append(df)

100%|██████████| 176/176 [00:30<00:00,  5.80it/s]


In [4]:
df_expiry_hist_data.tail()

,date,open,high,low,close,volume,oi,strike_type
624,2024-04-10 15:15:00+05:30,0.05,0.05,0.05,0.05,3225.0,179760.0,39000PE
625,2024-04-10 15:18:00+05:30,0.05,0.05,0.05,0.05,30.0,179760.0,39000PE
626,2024-04-10 15:21:00+05:30,0.05,0.05,0.05,0.05,2580.0,180480.0,39000PE
627,2024-04-10 15:24:00+05:30,0.05,0.05,0.05,0.05,900.0,179790.0,39000PE
628,2024-04-10 15:27:00+05:30,0.05,0.05,0.05,0.05,75.0,178890.0,39000PE


In [ ]:
# df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

In [13]:
# df_expiry_hist_data#.strike_type.nunique()

In [5]:
df_expiry_hist_data.to_parquet('../data/expiry_10aprail2024_3min.parquet')

## nifty

In [9]:
expiry_sp = 'NIFTY24410'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'NIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']
expiry_march.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,strike_type
558,9995522,39045,NIFTY2441022650CE,NIFTY,0.0,2024-04-10,22650.0,0.05,50,CE,NFO-OPT,NFO,22650CE
559,9996546,39049,NIFTY2441022650PE,NIFTY,0.0,2024-04-10,22650.0,0.05,50,PE,NFO-OPT,NFO,22650PE
560,9991938,39031,NIFTY2441022600CE,NIFTY,0.0,2024-04-10,22600.0,0.05,50,CE,NFO-OPT,NFO,22600CE
561,9992194,39032,NIFTY2441022600PE,NIFTY,0.0,2024-04-10,22600.0,0.05,50,PE,NFO-OPT,NFO,22600PE
562,9996802,39050,NIFTY2441022700CE,NIFTY,0.0,2024-04-10,22700.0,0.05,50,CE,NFO-OPT,NFO,22700CE


In [10]:
start_dt = '2024-04-04'
end_dt = '2024-04-10'
time_frame = '3minute'

df_expiry_hist_data = pd.DataFrame()
strike_symbol_dict = dict(zip(expiry_march.instrument_token,expiry_march['strike_type'] ))
inst_expiry = expiry_march['instrument_token'].unique().tolist() 
for i in tqdm(inst_expiry):
    df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
    df['strike_type'] = strike_symbol_dict[i]
    df_expiry_hist_data = df_expiry_hist_data.append(df)

df_expiry_hist_data#.strike_type.nunique()

100%|██████████| 176/176 [00:26<00:00,  6.75it/s]


,date,open,high,low,close,volume,oi,strike_type
0,2024-04-04 09:15:00+05:30,95.00,106.40,90.45,103.00,260300.0,326800.0,22650CE
1,2024-04-04 09:18:00+05:30,102.65,104.20,95.35,95.45,116100.0,323500.0,22650CE
2,2024-04-04 09:21:00+05:30,95.45,97.80,86.20,89.75,129300.0,327850.0,22650CE
3,2024-04-04 09:24:00+05:30,89.55,92.40,87.05,89.00,132000.0,340150.0,22650CE
4,2024-04-04 09:27:00+05:30,89.00,92.00,83.35,84.00,158700.0,353700.0,22650CE
...,...,...,...,...,...,...,...,...
618,2024-04-10 15:15:00+05:30,0.05,0.05,0.05,0.05,0.0,586000.0,20050PE
619,2024-04-10 15:18:00+05:30,0.05,0.05,0.05,0.05,50.0,586000.0,20050PE
620,2024-04-10 15:21:00+05:30,0.05,0.05,0.05,0.05,0.0,586000.0,20050PE
621,2024-04-10 15:24:00+05:30,0.05,0.05,0.05,0.05,0.0,586000.0,20050PE


In [11]:
df_expiry_hist_data.to_parquet('../data/nifty_expiry_10april2024_3min.parquet')